<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/Radar_SignalProc_Data_preprocessed_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1PDQk0c4hmB16BfbrWY3WU7K8vbN_Fmfb
!gdown --id 1ctUDilooaPt1pc4eEk-gyCMUcnRvI4aN
!gdown --id 1s5tXdlDm3mbtDCQs4Kl3VuLlUQ8J9MT3


/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1PDQk0c4hmB16BfbrWY3WU7K8vbN_Fmfb
To: /content/nobody.npy
100% 13.7M/13.7M [00:00<00:00, 130MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ctUDilooaPt1pc4eEk-gyCMUcnRvI4aN
To: /content/stand.npy
100% 13.7M/13.7M [00:00<00:00, 48.1MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1s5

In [ ]:
import numpy as np 
nobody   = np.load("/content/nobody.npy")
standing = np.load("/content/stand.npy")
walking  = np.load("/content/walk.npy")
print(nobody.shape)
print(standing.shape)
print(walking.shape)
print("60 second of record with 255 FPS")
print("60 second x 255 frame per second ="+str(60*255))

(15300, 112)
(15300, 112)
(15300, 112)
60 second of record with 255 FPS
60 second x 255 frame per second =15300


In [ ]:
# I/Q data reader 
def IQdata(datas):
    iq_vecs = []
    for data in datas:
        data_length = len(data)
        i_vec = np.array(data[:data_length//2])
        q_vec = np.array(data[data_length//2:])
        iq_vec = i_vec + 1j*q_vec
        iq_vecs.append(iq_vec)
    return np.stack(iq_vecs)

IQ_nobody   = IQdata(nobody)
IQ_standing = IQdata(standing)
IQ_walking  = IQdata(walking)

print(IQ_nobody.shape)
print(IQ_standing.shape)
print(IQ_walking.shape)
print("56 Range bits of 2.8 meter of distance")

(15300, 56)
(15300, 56)
(15300, 56)
56 Range bits of 2.8 meter of distance


In [ ]:
#Range time data
def range_time(IQ_data):
    n_rd_history = 255
    frame = []
    frames = []

    for iqini in IQ_data:
        if len(frame)<n_rd_history:
            frame.append(iqini)
        else:  
            frames.append(np.copy(frame))
            frame.append(iqini)
            frame = frame[1::]
            
    return np.stack(frames)

Range_time_nobody   = range_time(IQ_nobody)
Range_time_standing = range_time(IQ_standing)
Range_time_walking  = range_time(IQ_walking)

print(Range_time_nobody.shape)
print(Range_time_standing.shape)
print(Range_time_walking.shape)

(15045, 255, 56)
(15045, 255, 56)
(15045, 255, 56)


In [ ]:
import scipy.fftpack as fft
#Range frequency data
def Range_frequency(datas):
    Range_frequency_frame = []
    for data in datas:
        dB = True
        # Range-Doppler
        rd = fft.fft(data, axis=0)
        rd = fft.fftshift(rd, axes=0)
        rd = abs(rd)
        DBrd = 20 * np.log10(rd+1e-10)
        Range_frequency_frame.append(DBrd)
    return np.stack(Range_frequency_frame)

Range_frequency_nobody   = Range_frequency(Range_time_nobody)
Range_frequency_standing = Range_frequency(Range_time_standing)
Range_frequency_walking  = Range_frequency(Range_time_walking)

print(Range_frequency_nobody.shape)
print(Range_frequency_standing.shape)
print(Range_frequency_walking.shape)

(15045, 255, 56)
(15045, 255, 56)
(15045, 255, 56)
